# Oxidation of oxygen

In this notebook we'll take a look at the participation of oxygen in the redox reactions of the battery as it is charged. Here's a brief overview of the plan:

#### Magnetic moments
1. Check state of oxidation of the substituted element from the magnetic moments
2. Calculate the average magnetic moment of oxygen around the substituted element.
3. Calculate the average magnetic moment of the other oxygens and compare.

#### PDOS
1. Check the energy levels of the substituted element versus the oxygen in the discharged structure.
2. Compare the PDOS before and after charging for oxygen.


In [1]:
# Notebook header <---- Execute this cell first!

import os, json
import numpy as np

from monty.json import MontyDecoder
from ipywidgets import interact, FloatSlider, FloatRangeSlider
from pybat.core import Cathode, VORONOI_ANG_FACTOR, VORONOI_DIST_FACTOR

from pymatgen.electronic_structure.core import Orbital, OrbitalType
from pymatgen.electronic_structure.dos import Dos, CompleteDos
from pymatgen.electronic_structure.plotter import DosPlotter
from pymatgen import Element, Spin

# Data being considered - Will be placed in data subdirectory
doping_elements = ["Co", "Mo", "V", "Sn"]
states = ["discharged", "charged"]
functionals = ["scan", "pbeu"]

## Magnetic Moments

In [2]:
with open("data/magmom_data.json", "r") as file:
    magmom_data = json.loads(file.read())

# Function for interactive analysis
def analyze_magmoms(doping_element, state, functional):
    
    magmom_dict = magmom_data[doping_element][state][functional]
    
    print("Magnetic moment of " + doping_element + ": " 
          + str(round(magmom_dict["dopant"], 3)))
    print("Average magnetic moment of neighbouring oxygen atoms: " + 
         str(round(magmom_dict["neighbor_O"], 3)))
    print("Average magnetic moment of other oxygen atoms in the structure: " + 
         str(round(magmom_dict["other_O"], 3)))

In [3]:
interact(analyze_magmoms,
         doping_element=doping_elements, 
         state=states, functional=functionals);

interactive(children=(Dropdown(description='doping_element', options=('Co', 'Mo', 'V', 'Sn'), value='Co'), Dro…

## PDOS

In [4]:
with open("data/pdos_data.json", "r") as file:
    pdos_data = json.loads(file.read())

# Method for quickly plotting the element-projected DOS
def plot_el_total_dos(complete_dos, elements, smearing_sigma=0.05, xlim=None, ylim=None):
    """
    
    Args:
        complete_dos (CompleteDos):
        elements: list or tuple of string representations of the elements, e.g. ["Mn", "O"]
    
    """
    
    elements_dos = complete_dos.get_element_dos()

    total_dos_dict = dict()
    
    for el in elements:
        
        el_dos = elements_dos[Element(el)]
        
        total_dos_dict[el] = Dos(
            efermi=el_dos.efermi, energies=el_dos.energies,
            densities={Spin.up:el_dos.get_smeared_densities(sigma=smearing_sigma)[Spin.up] 
                       + el_dos.get_smeared_densities(sigma=smearing_sigma)[Spin.down]}
        )
        
    plotter = DosPlotter()
    plotter.add_dos_dict(total_dos_dict)
    plotter.show(xlim=xlim, ylim=ylim)

# Function for the interactive analysis of the PDOS
def analyze_pdos(doping_element, state, functional, xlim=None, ymax=200):
    
    cdos = CompleteDos.from_dict(
        pdos_data[doping_element][state][functional]
    )
    
    plot_el_total_dos(
        complete_dos=cdos,
        elements = [doping_element, "Mn", "O"],
        xlim=xlim,
        ylim=(0, ymax)
    )

In [5]:
interact(analyze_pdos, doping_element=doping_elements, state=states,
         functional=functionals,
         xlim=FloatRangeSlider(value=(-10, 5), min=-20, max=10),
         ymax=FloatSlider(value=150, min=0, max=300));

interactive(children=(Dropdown(description='doping_element', options=('Co', 'Mo', 'V', 'Sn'), value='Co'), Dro…